In [1]:
"""
@filename rescale.py

Rescale the wavelength solution onto the constant delta lamba over lambda scale
of the reference library. Resample the spectrum over these wavelength values.
"""

import sys
import os 
#from deblaze import afs_deblaze #ADZ comment out 6/25
import pandas as pd
import csv
from pylab import *
import astropy.io.fits as pf
sys.path.append('specmatch-emp')
import specmatchemp.library
import specmatchemp.plots as smplot
from specmatchemp import spectrum
from specmatchemp.specmatch import SpecMatch

def get_rescaled_wave_soln():
    """
    Acquire a rescaled wavelength array on the scale of the reference library,
    focused on the range 4997-6338 Å

    Returns:
        wave_solution (numpy.ndarray): Wavelength array rescaled on the constant
        wavelength scale of the library and focused on the range 4997-6338 Å
    """
 
    v = 1.017454512 # [km/s] - pixel spacing
    c = 2.99792458e+05 # [km/s] - speed of light

    # block print messages - intended for the function "library.read_hdf()"
    old_stdout = sys.stdout #ADZ ADD 6/24/20 to save a reference to the original standard output to restore later
    sys.stdout = open(os.devnull, 'w')
    
    
    lib_temp = specmatchemp.library.read_hdf()
    
    
    # restore print messages
    #sys.stdout = sys.__stdout__ #ADZ COMMENT 6/24 This is not restoring output!
    sys.stdout = old_stdout #ADZ ADD 6/24
    
    # use random reference star to acquire correctly scaled wave solution
    # w = wavelength
    # s = flux
    

    idx1 = lib_temp.get_index('190406')
    G_star = lib_temp.pop(idx1)
    w_ref = G_star[1].w

    # SME wave soln is on constant delta lambda over lambda scale
    # using this scale, we create an array of wavelengths values
    # that extends the range of the reference wavelength array
    # (APF spectra cover more wavelength range than the reference library)

    # calculate values lower than the lowest reference value
    # 3729.77... = lowest wavelength value in APF wavelength solution
    w_lower = []
    w = 5000      # arbitrary high value to pass into loop
    current = w_ref[0]
    while (w >= 3729.7756515574665):
        w = current / (1+v/c)
        current = w
        w_lower.append(w)
    w_lower.reverse()
    w_lower = w_lower[1:]
    

    # calculate values higher than the highest reference value
    # 10215.33... = highest wavelength value in APF wavelength solution
    w_higher = []
    w = 0
    current = w_ref[-1]
    while (w <= 10215.333386992712):
        w = current * (1+v/c)
        current = w
        w_higher.append(w)
    w_higher = w_higher[:-1]

    # combine all the values of w_lower, w_ref, w_higher
    wave_solution = []
    for e in w_lower:
        wave_solution.append(e)
    for e in w_ref:
        wave_solution.append(e)
    for e in w_higher:
        wave_solution.append(e)

    # Focus onto the region 4997-6338 Å, which *by eye* seems to be the region
    # that shifting performs best in
    # note: this region can be refined -> future work
    wave_solution = wave_solution[86188:156208]
    
    print('Finished rescale') #ADZ ADD

    return wave_solution

def resample(wave_data_new, wave_data_old, data):

    """
    Resample a spectrum onto a new wavelength solution
    by averaging intensity values within +/- 0.017 Å on the new wavelength array
    (typically an average of two intesity values for each wavelength,
    with exception to regions where spectral orders overlap in which case
    there is an average of more intensity values per wavelength)
    Args:
        wave_data_new (numpy.ndarray): Wavelength array used
        to resample the data
        wave_data_old (numpy.ndarray): Initial wavelength array corresponding
        to the data
        data (numpy.ndarray): Initial intensity data
    Returns:
        data_new (numpy.ndarray): New intensity data that corresponds to the
        new wavelength solution
    """

    # Focus data onto the wavelength region 4997-6338 Å
    # index 3055 in order 31 roughly corresponds to 5000 Å
    # index 3642 in order 52 roughly corresponds to 6338 Å
    # note: here 30 actually corresponds to spectral order 31,
    # due to python starting indexing at 0,
    # so for reference 0 is order 1, and 1 is order 2, etc.

    # w_lst = wavelength list
    # s_lst = intensity list

    w_lst = []
    s_lst = []
    w_lst.append(wave_data_old[30, 3055:4607].tolist())
    s_lst.append(data[30, 3055:4607].tolist())
    for i in range(20):
        w_lst.append(wave_data_old[31 + i, :4607].tolist())
        s_lst.append(data[31 + i, :4607].tolist())
    w_lst.append(wave_data_old[51, :3642].tolist())
    s_lst.append(data[51, :3642].tolist())
    w_lst = [item for sublist in w_lst for item in sublist]
    s_lst = [item for sublist in s_lst for item in sublist]
    

    # create array that has wavelengths in column 0 and intensity in column 1
    ws = np.zeros((len(w_lst), 2))
    for i in range(len(w_lst)):
        ws[i,0] = w_lst[i]
        ws[i,1] = s_lst[i]

    # sort the array by increasing wavelength
    # the result of merging spectral orders does not strictly increase
    # because orders overlap in wavelength ranges
    ws = ws[ws[:, 0].argsort()]

    # select to skip first 137 values (closer to 4997 Å) - via eye test
    ws = ws[137:]

    # average intensity values within +/- 0.017 Å on the new wavelength scale
    
    # comparisons between wavelengths in the old and new scale start from the
    # beginning of the old wavelength array for each new wavelength value
    # inefficient, so data points in the front of the array are clipped every
    # 1000 comparisons to reduce time wasted on wavelength comparisons
    data_new = []
    count = 0
    pairs = ws.tolist() # pairs -> (old wavelength, old intensity)
    for w in wave_data_new: # for every new wavelength value
        if (count != 0):
            if ((count % 1000 == 0)): # for every thousand comparisons

                # remove 800 pairs from the beginning of the
                # list of old values
                removed_1 = pairs[:800]
                removed_1.reverse()
                del pairs[:800]

                # if we remove too many,
                # the new wavelength value is less than the
                # starting wavelength value in the old list
                # i.e. (w_new - w_old < 0)
                # so we return some of the pairs
                if (w - pairs[0][0] < 0):
                    for pair in removed_1:
                        pairs.insert(0, pair)
                        break # currently breaks after inserting one pair

                # while the new wavelength value is greater than
                # the starting wavelength in the old list by more than a tenth,
                # remove a pair
                # (to improve efficiency, avoiding unneccessary comparisons)
                while ((w - pairs[0][0] > .1)):
                    removed_2 = pairs[0]
                    del pairs[0]
                    if (w - pairs[0][0] < 0): # replace if (w_new - w_old < 0)
                        pairs.insert(0, removed_2)
                        break
        avg_lst = []
        some_found = False
        
        for pair in pairs:
            # comparison process between new and old wavelength values
            if (abs(w - pair[0]) < .017):
                avg_lst.append(pair[1])
                some_found = True
            elif (some_found): break # break when the inequality no longer holds
        data_new.append(np.mean(np.asarray(avg_lst)))
        count += 1

        
    print('Finished resample') #ADZ DELETE THIS 6/23/20
    return data_new

ModuleNotFoundError: No module named 'deblaze'